# Avaliação 3 – Risco
## Construção de uma calculadora de retornos e VaR em Python

Data prevista de entrega: 10/01/2022

Formato de envio: arquivo .py conforme instruções a seguir.
- Pode fazer orientado a objetos ou funcional;
- Pode construir quantos métodos/classes/funções adicionais desejarem;
- Pode utilizar bibliotecas abertas;
- O arquivo deve conter as funções solicitadas e os cenários de teste (pelo menos um teste para cada função/método);
- Comente as funções e o que mais achar necessário no código;

A nota da avaliação considerará apenas o resultado das funções desejadas (existência de bugs, bom funcionamento, etc);<br>
Na avaliação qualitativa, faremos comentários de melhorias na estrutura do código (para fins de evolução de aprendizado, e não para reduzir nota).<br>

### Funções/métodos mínimos de entrega:
- calcula_retorno(pandas serie_precos, holding_period)
    - Retorna: retorno simples (pd.Dataframe)


- calcula_log_retorno(pandas serie_precos, holding_period)
    - Retorna: log-retorno (pd.Dataframe)


- calcula_var(pandas serie_retornos, alpha, vlr_carteira_atual)
    - Retorna VaR (float)


- calcula_es(pandas serie_retornos, alpha, vlr_carteira_atual)


## Bibliotecas

In [1]:
# Manipulação de dados
import numpy as np
import pandas as pd
#import datetime as dt
from datetime import datetime

# Graficos
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# Calculos
from scipy.stats import norm

# Saida de dados tabular
from tabulate import tabulate 

# Yahoo Finance
import yfinance as yf


## Funções

In [2]:
def carrega_acao(acao, dt_inicio, dt_fim):
    """
    Carrega dataFrame com cotações da ação
    
    Parametros
        acao (string)
        
        dt_inicio (string): "aaaa-mm-dd"
        Onde, aaaa = ano
              mm = mes
              dd = dia
        
        dt_fim (string): "aaaa-mm-dd"
        Onde, aaaa = ano
              mm = mes
              dd = dia
              
    Retorno
        dataFrame
        Retorna dataFrame com a precificação da ação no período informado.
    
    Exemplo
        >>> carrega_acao(acao, dt_inicio, dt_fim)
    """
    try:
        dt_inicio = datetime.strptime(dt_inicio, '%Y-%m-%d').date()
        dt_fim = datetime.strptime(dt_fim, '%Y-%m-%d').date()

        df = yf.download(acao, dt_inicio, dt_fim) 

        return df

    except Exception as exception:
        print("Exception: {}".format(type(exception).__name__))
        print("Exception message: {}".format(exception))


In [3]:
def calcula_retorno(df, col_name, hp):
    """
    Calculo de Retorno e PnL (Profit and Loss / Lucros e Perdas)
    
    Parametros
        df (dataFrame)
        
        col_name (string)
        Nome da coluna de preço para cálculo de Retorno e PnL 
        
        hp (int)
        Holding period / Período de retenção
        Onde, 1 dia usado para cáculo de limites
              10 dias usado para cálculo de capital
              30 dias usado em métricas de liquidez
    
    Retorno
        dataFrame
        Retorna o dataFrame de origem com as colunas [return] e [PnL] .
    
    Exemplo
        >>> calcula_retorno(df, col_name, hp)
    """
    try:    
        # Calculo do Retorno usando cálculo manual
        df['Return'] = (df[col_name] / df[col_name].shift(hp) - 1) # Retorno

        # Calculo do PnL
        df['PnL'] = df[col_name] - df[col_name].shift(hp) # Realizado

        # Calculo do Retorno usando função do Pandas (pct_change)
        # Calcula o percentual entre o elemento corrente e um elemento anterior por padrão.
        # Isto é útil para comparar a percentagem de mudança numa série temporal de elementos.
        # df['Return_pd'] = df[col_name].pct_change(periods=hp)

        return df

    except Exception as exception:
        print("Exception: {}".format(type(exception).__name__))
        print("Exception message: {}".format(exception))
    

In [4]:

def calcula_log_retorno(df, col_name, hp):
    """
    Calculo de Log Retorno
    
    Parametros
        df (dataFrame)
        
        col_name (string)
        Nome da coluna da valor retorno para cálculo LOG
        
        hp (int)
        Holding period / Período de retenção
        Onde, 1 dia usado para cáculo de limites
              10 dias usado para cálculo de capital
              30 dias usado em métricas de liquidez
    
    Retorno
        dataFrame
        Retorna o dataFrame de origem com a coluna [logReturn]
    
    Exemplo
        >>> calcula_log_retorno(df, col_name, hp)
    """
    try:    
        df['logReturn'] = df.apply(lambda x: np.log(x[col_name]), axis=1) # log retorno    
        
        return df

    except Exception as exception:
        print("Exception: {}".format(type(exception).__name__))
        print("Exception message: {}".format(exception))


In [5]:

def calcula_var(df, col_name, alpha):
    """
    Calcula o VaR - Value at Risk
    
    Parametros
        df (dataFrame)
        
        col_name (string)
        Nome da coluna de cálculo de Retorno
        
        alpha (float)
        Nível de confiança
        Onde, 0.1  indica um grau de confiança de 90%
              0.05 indica um grau de confiança de 95%
              0.01 indica um grau de confiança de 99%
    
    Retorno
        VaR (float), mean (float), std_dev (float)
    
    Exemplo
        >>> calcula_var(df, col_name, alpha)    
    """
    try:
        # Calcula a média
        mean = np.mean(df[col_name])

        # Calcula a volatilidade
        std_dev = np.std(df[col_name])

        VaR = norm.ppf(alpha, mean, std_dev)

        return VaR, mean, std_dev
    
    except Exception as exception:
        print("Exception: {}".format(type(exception).__name__))
        print("Exception message: {}".format(exception))


In [6]:

def calcula_var_hist(df, col_name, alpha):
    """
    Calcula o VaR Histórico - Value at Risk
    
    Parametros
        df (dataFrame)
        
        col_name (string)
        Nome da coluna de cálculo de Retorno
        
        alpha (float)
        Nível de confiança
        Onde, 0.1  indica um grau de confiança de 90%
              0.05 indica um grau de confiança de 95%
              0.01 indica um grau de confiança de 99%
    
    Retorno
        VaR_hist (float)
    
    Exemplo
        >>> calcula_var_hist(df, col_name, alpha)    
    """
    try:
        # Calcula o VaR histórico
        VaR_hist = df[col_name].quantile(alpha)

        return VaR_hist
    
    except Exception as exception:
        print("Exception: {}".format(type(exception).__name__))
        print("Exception message: {}".format(exception))


In [7]:

def calcula_es(df, col_name, alpha, VaR_hist=0):
    """
    Calcula o Valor Esperado - Expected shortfall (ES)
    
    Parametros
        df (dataFrame)
        
        col_name (string)
        Nome da coluna para calculo
        
        alpha (float)
        Nível de confiança 
        Onde, 0.1  indica um grau de confiança de 90%
              0.05 indica um grau de confiança de 95%
              0.01 indica um grau de confiança de 99%
              
        VaR_hist (float)
        Onde, 0 indica que o VaR histórico precisa ser calculado, caso o valor seja diferente de 0
              será usado como valor do VaR histórico 
    
    Retorno
        es (float)
    
    Exemplo
        >>> calcula_es(df, col_name, alpha, VaR_hist=0)
    """
    try:
        if VaR_hist == 0:
            VaR_hist_calc = calcula_var_hist(df, col_name, alpha)
        else:
            VaR_hist_calc = VaR_hist

        amostra_es = df[col_name][df[col_name] <= VaR_hist_calc]
        es = amostra_es.mean()

        return es
    
    except Exception as exception:
        print("Exception: {}".format(type(exception).__name__))
        print("Exception message: {}".format(exception))


## Testes

### Carrega Carteira

In [52]:
# Carrega dataFrame com os prços da ação desejada

#df = carrega_acao("ITSA4.SA", "2019-01-01", "2022-12-31")
df = carrega_acao("PETR4.SA", "2019-01-01", "2022-12-31")

#df = carrega_acao("MSFT", "2019-01-01", "2022-12-31")
#df = carrega_acao("AAPL", "2019-01-01", "2022-12-31")

#df = carrega_acao("ETH-USD", "2019-01-01", "2022-12-31")
#df = carrega_acao("BTC-USD", "2016-01-01", "2022-12-31")


[*********************100%***********************]  1 of 1 completed


### Calcula Retorno e PnL da Carteira

In [53]:
col_name = 'Close'
hp = 1
calcula_retorno(df, col_name, hp)
df

#df.reset_index(inplace=True)
#df.set_index("Date",inplace=True)
#df.dropna(inplace=True)

,Open,High,Low,Close,Adj Close,Volume,Return,PnL
Date,,,,,,,,
2019-01-02,22.549999,24.200001,22.280001,24.059999,18.925686,104534800,NaN,NaN
2019-01-03,23.959999,24.820000,23.799999,24.650000,19.389780,95206400,0.024522,0.590000
2019-01-04,24.850000,24.940001,24.469999,24.719999,19.444843,72119800,0.002840,0.070000
2019-01-07,24.850000,25.920000,24.700001,25.110001,19.751619,121711900,0.015777,0.390001
2019-01-08,25.400000,25.420000,24.770000,24.959999,19.633627,68761800,-0.005974,-0.150002
...,...,...,...,...,...,...,...,...
2022-02-14,33.860001,33.939999,32.820000,33.000000,33.000000,55687000,-0.022512,-0.759998
2022-02-15,32.660000,32.680000,31.889999,32.480000,32.480000,65689000,-0.015758,-0.520000
2022-02-16,32.830002,33.700001,32.770000,32.930000,32.930000,65811600,0.013855,0.450001


### Calcula o Value at Risk


In [54]:
# variaveis de nível de confiança
conf_90  = 0.1   # 1 - 0.9
conf_95  = 0.05  # 1 - 0.95
conf_975 = 0.025 # 1 - 0.975
conf_99  = 0.01  # 1 - 0.99

col_name = "PnL"
VaR_90,  Med, Vol = calcula_var(df, col_name, conf_90)
VaR_975, Med, Vol = calcula_var(df, col_name, conf_975)
VaR_95,  Med, Vol = calcula_var(df, col_name, conf_95)
VaR_99,  Med, Vol = calcula_var(df, col_name, conf_99)


print(tabulate([['90.0%', VaR_90], ['95.0%', VaR_95], ['97.5%', VaR_975], ["99.0%", VaR_99]], headers=['Nível de Confiança', 'Value at Risk']))
    

Nível de Confiança      Value at Risk
--------------------  ---------------
90.0%                       -0.877567
95.0%                       -1.12961
97.5%                       -1.34821
99.0%                       -1.60239


### Calcula Value at Risk Histórico


In [55]:
col_name = "PnL"
VaRh_90  = calcula_var_hist(df, col_name, alpha=conf_90)
VaRh_95  = calcula_var_hist(df, col_name, alpha=conf_95)
VaRh_975 = calcula_var_hist(df, col_name, alpha=conf_975)
VaRh_99  = calcula_var_hist(df, col_name, alpha=conf_99)

print(tabulate([['90.0%', VaRh_90], ['95.0%', VaRh_95], ['97.5%', VaRh_975], ["99.0%", VaRh_99]], headers=['Nível de Confiança', 'Value at Risk']))

Nível de Confiança      Value at Risk
--------------------  ---------------
90.0%                       -0.654
95.0%                       -0.974001
97.5%                       -1.29
99.0%                       -1.8412


### Calcula Expected Shortfall (ES)

In [56]:
col_name = "PnL"
es_90 = calcula_es(df, col_name, conf_90, VaRh_90)
es_95 = calcula_es(df, col_name, conf_95, VaRh_95)
es_975 = calcula_es(df, col_name, conf_975, VaRh_975)
es_99 = calcula_es(df, col_name, conf_99, VaRh_95)

print(tabulate([['90.0%', es_90], ['95.0%', es_95], ['97.5%', es_975], ["99.0%", es_99]], headers=['Nível de Confiança', 'Expected Shortfall']))


Nível de Confiança      Expected Shortfall
--------------------  --------------------
90.0%                             -1.25705
95.0%                             -1.73154
97.5%                             -2.24571
99.0%                             -1.73154


In [57]:
# 
VaR_List = []
VaR_List.append(("90%",   VaR_90,  VaRh_90,  es_90 ))
VaR_List.append(("95%",   VaR_95,  VaRh_95,  es_95 ))
VaR_List.append(("97.5%", VaR_975, VaRh_975, es_975))
VaR_List.append(("99%",   VaR_99,  VaRh_99,  es_99 ))

column_names = ["Nivel de Confiança", "VaR", "VaR Hist", "ES"]
VaR_pd = pd.DataFrame (VaR_List, columns = column_names)

cols = VaR_pd.select_dtypes([np.number]).columns
VaR_pd[["Nivel de Confiança"]].join(VaR_pd[cols] * -1)

,Nivel de Confiança,VaR,VaR Hist,ES
0,90%,0.877567,0.654000,1.257051
1,95%,1.129607,0.974001,1.731538
2,97.5%,1.348214,1.289999,2.245714
3,99%,1.602391,1.841200,1.731538
